In [1]:
import numpy as np
import pandas as pd
from mm1 import *
import matplotlib.pyplot as plt

In [2]:
mm1 = MM1(None, 10, 15)

In [169]:
class SimulacaoMM1:
    def __init__(self, chegada, atendimento, distribuicao = 'Exponencial'):
        self.chegada = chegada
        self.atendimento = atendimento
        self.distribuicao = distribuicao
        self.simulacaoMM1 = gerarTabelaSimulacaoMM1(self.chegada, self.atendimento, self.distribuicao)
        self.mm1 = MM1(None, self.chegada, self.atendimento)
        self.mm1.calcularTudo()
    
    def gerarDistribuicaoNumpy(self, parametro = 1):
        # Retorna uma variável da distribuição selecionada
        if self.distribuicao == 'Exponencial':
            return np.random.exponential(1/parametro)

        elif self.distribuicao == 'Normal':
            return abs(np.random.normal())

        elif self.distribuicao == 'Uniforme':
            return abs(np.random.uniform())

        else:
            raise Exception('Distribuição não reconhecida/disponibilizada')

    def gerarTabelaSimulacaoMM1(self):
        # M/M/1
        i = 1
        tcr = 0
        tsr = 0

        # TCR/TSR: tempo chegada/saída no relógio, TF: tempo de fila, TAO: tempo atendente ocioso
        # TClS: tempo cliente no sistema
        data = {'ID': [], 'TEC': [], 'TCR': [], 'TES': [], 'TF':[], 'TSR': [], 'TClS': [], 'TAO': []}

        #while i <= 1000:
        while tcr <= 1:
            tec = gerarDistribuicaoNumpy(self.chegada)
            tes = gerarDistribuicaoNumpy(self.atendimento)
            tcr = tcr + tec

            tf = max(0, tsr - tcr)
            tao = max(0, tcr - tsr)

            if tcr < tsr: tsr = tsr + tes + tf
            else: tsr = tcr + tes

            tcls = tf + tes

            data['ID'].append(i)
            data['TEC'].append(tec)
            data['TES'].append(tes)
            data['TCR'].append(tcr)
            data['TSR'].append(tsr)
            data['TF'].append(tf)
            data['TAO'].append(tao)
            data['TClS'].append(tcls)

            i = i + 1

        return pd.DataFrame(data)

    def gerarTabelaComparacao(self):
        data = {
            'TIPO': ['Simulação M/M/1', 'Modelo M/M/1'],
            'W': [self.simulacaoMM1['TClS'].mean(), self.mm1.l],
            'Wq': [self.simulacaoMM1['TF'].mean(), self.mm1.wq],
            'Ws': [self.simulacaoMM1['TClS'].mean() - self.simulacaoMM1['TF'].mean(), self.mm1.ws],
            'L': [None, self.mm1.l],
            'Lq': [None, self.mm1.lq],
            'Ls': [None, self.mm1.ls]
        }

        return pd.DataFrame(data)

In [190]:
s = SimulacaoMM1(10, 15)
s.gerarTabelaComparacao()

,TIPO,W,Wq,Ws,L,Lq,Ls
0,Simulação M/M/1,2.721762,2.621658,0.100104,NaN,NaN,NaN
1,Modelo M/M/1,2.000000,0.133333,0.066667,2.0,1.333333,0.666667


,TIPO,W,Wq,Ws,L,Lq,Ls
0,Simulação M/M/1,0.122827,0.068136,0.054691,NaN,NaN,NaN
1,Modelo M/M/1,2.000000,0.133333,0.066667,2.0,1.333333,0.666667


In [124]:
gerarTabelaComparacao(10, 15)

,TIPO,W,Wq,Ws,L,Lq,Ls
0,Simulação M/M/1,2.166797,2.094326,0.072471,NaN,NaN,NaN
1,Modelo M/M/1,2.000000,0.133333,0.066667,2.0,1.333333,0.666667


In [153]:
df = gerarTabelaSimulacaoMM1(10,15, 'Exponencial')
df.head()

2.3295986216980507

In [155]:
df

,ID,TEC,TCR,TES,TF,TSR,TClS,TAO
0,1,0.006189,0.006189,0.010626,0.000000,0.016816,0.010626,0.006189
1,2,0.093888,0.100077,0.138274,0.000000,0.238351,0.138274,0.083262
2,3,0.069981,0.170058,0.000723,0.068293,0.307366,0.069015,0.000000
3,4,0.033731,0.203789,0.021051,0.103577,0.431995,0.124628,0.000000
4,5,0.143685,0.347474,0.046828,0.084521,0.563343,0.131349,0.000000
5,6,0.041094,0.388568,0.018730,0.174775,0.756849,0.193506,0.000000
6,7,0.086346,0.474914,0.030261,0.281935,1.069045,0.312196,0.000000
7,8,0.146582,0.621495,0.014229,0.447550,1.530824,0.461779,0.000000
8,9,0.043132,0.664627,0.191270,0.866196,2.588290,1.057466,0.000000
9,10,0.095037,0.759664,0.053048,1.828626,4.469963,1.881674,0.000000


In [82]:
df[df.TF > 0].shape[0]

993

In [79]:
993/1000

0.993